In [271]:
import graphlab
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

pd.set_option("display.max_colwidth", 1000)

In [272]:
jokes_contents = pd.read_table("data/jokes.dat")     
jokes_contents.head()


,1:
0,<p>
1,"A man visits the doctor. The doctor says, &quot;I have bad news for you. You have cancer and Alzheimer&#039;s disease&quot;.<br />"
2,<br />
3,"The man replies, &quot;Well, thank God I don&#039;t have cancer!&quot;"
4,</p>


In [273]:
# Training data
ratings_contents = pd.read_table("data/ratings.dat")    
ratings_contents.head()
# ratings_contents.shape

,user_id,joke_id,rating
0,34888,111,-9.71875
1,49651,119,4.75000
2,29027,137,2.62500
3,25235,30,-2.43750
4,50499,19,2.18750


# Notes/Observations
- Ratings range from -10 to +10
- Explicit data because of ratings
- Use a user-item based utility matrix
- Jokes 7, 8, 13, 15, 16, 17, 18, 19 are the gauge set
- Jokes 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 20, 27, 31, 43, 51, 52, 61, 73, 80, 100, 116 have been removed.

Questions:
- What does the gauge set mean? 


1. Need to construct a utility matrix of users vs. items

In [274]:
sample_submissions_contents = pd.read_csv("data/sample_submission.csv")
sample_submissions_contents

sample_submissions_contents.shape
sample_submissions_contents.head()

,user_id,joke_id,rating
0,49541,113,5
1,39499,37,5
2,36804,73,5
3,16470,33,5
4,22614,128,5


# Notes/Observations
- Half the size of the ratings_contents data because it takes out users who did not answer the gauge set.


Questions:
- What does the gauge set mean? 

In [275]:
sample_submissions_contents = pd.read_csv("data/test_ratings.csv")
sample_submissions_contents.head()

,user_id,joke_id,rating
0,49541,113,4.010419
1,39499,37,1.621052
2,36804,73,5.072449
3,16470,33,6.214692
4,22614,128,5.999286


In [276]:
jokes_contents = pd.read_table("data/jokes.dat")     
jokes_contents.head()
#jokes_string = jokes_contents['1:'].apply(lambda x: ' '.join(x))

# Combine all rows in jokes_content into one string
joke_string = jokes_contents['1:'].str.cat(sep=' ')

# Separate each joke into individual strings
joke_string_split = joke_string.split('</p')

# Create a df with each joke
joke_df = pd.DataFrame(joke_string_split)

for joke in joke_df:
    joke_df['Joke ID'] = joke_df.index + 1

cols = joke_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
joke_df = joke_df[cols]
joke_df.head()

# Slice of meaningless characters in jokes

type(joke_df)

pandas.core.frame.DataFrame

In [ ]:
joke_df.rename(columns={0:'Joke Text'}, inplace=True)

joke_df.head()
joke_df['Joke Text'] = joke_df['Joke Text'].map(lambda x: x.lstrip('<>p').rstrip('>'))
joke_df['Joke Text'] = joke_df['Joke Text'].map(lambda x: x.lstrip('<p>').rstrip('>'))
joke_df['Joke Text'] = joke_df['Joke Text'].str.replace('\d+', '')
joke_df['Joke Text'] = joke_df['Joke Text'].str.replace(':', '')
joke_df['Joke Text'] = joke_df['Joke Text'].str.replace('<p>', '')
joke_df['Joke Text'] = joke_df['Joke Text'].str.replace('Q.', '')
joke_df = joke_df.replace({'&#;': '', '<br />': '', '&quot;':''}, regex=True)

joke_df.head()

# Join the new joke text with old dataframe

new_joke_df = pd.concat([joke_df, ratings_contents])

ratings_contents.head()
new_joke_df = joke_df.merge(ratings_contents, left_on='Joke ID', right_on='joke_id', how='outer')

del new_joke_df['joke_id']

new_joke_df.head()
# count = new_joke_df['Joke Text'].str.split().apply(len).value_counts()
new_joke_df['Joke Text'].str.split()

# Different things to text for NLP
- NMF

New SideData:
- Topics each joke belongs to a specific number
- # of words in a joke
- # Key words within the highest rated jokes, add % of words 
- Stemming & Lemmatizing


In [ ]:
for joke in np.unique(new_joke_df["Joke Text"]):
    new_joke_df[new_joke_df['Joke Text'] == joke]['Word Counts'] = len(joke)
    
new_joke_df.head()